In [1]:
import pandas as pd
import csv
import numpy as np

In [121]:
first_data = pd.read_csv('../../1995_Street_Tree_Census.csv')
second_data = pd.read_csv('../../2005_Street_Tree_Census.csv')
third_data = pd.read_csv('../../2015_Street_Tree_Census.csv')

/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [122]:
# Extract the columns we care about
first_data = first_data[["Latitude", "Longitude", "Borough", "Zip Codes", "Spc_Latin", "Diameter", "Condition"]]
second_data = second_data[["latitude", "longitude", "boroname", "zipcode", "spc_latin", "tree_dbh", "status"]]
third_data = third_data[["latitude", "longitude", "borough", "postcode", "spc_latin", "tree_dbh", "status", "health"]]

# Standardize all health statuses across the 2005 and 2015 datasets

In [123]:

first_data
first_data.dtypes
first_data.Condition.unique()
# first_data.Borough.unique()

array(['Unknown', 'Good', 'Excellent', 'Poor', 'Dead', 'Stump',
       'Planting Space', 'Shaft', 'Fair', 'Critical'], dtype=object)

In [124]:
# Rename "Excellent" to "Good", rename "Good" to "Fair"
second_data.status[second_data['status'] == "Good"] = "Fair"
second_data.status[second_data['status'] == "Excellent"] = "Good"
second_data = second_data.rename(columns={"status": "health"})


/var/folders/z7/x5j_gv2x6f92wlfl832cwwy80000gn/T/ipykernel_85224/1737469685.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_data.status[second_data['status'] == "Excellent"] = "Good"


array(['Fair', 'Poor', 'Good', 'Dead'], dtype=object)

In [125]:
# Set all status to be "Alive" or "Dead" (remove "Stump")
third_data.health[(third_data['health'].isna()) & (third_data['status'] != "Alive")] = "Dead"
# TODO remove the one alive and health is na row

# third_data.dtypes
third_data.health.unique()
# third_data.status.unique()
# third_data.borough.unique()
# third_data.root_stone


array(['Fair', 'Good', 'Poor', 'Dead', nan], dtype=object)

In [127]:
# Group trees into geospatial segments by lat and long
step = 0.00000001 # This should be about 1/4-1/2 a city block 
to_bin = lambda x: np.floor(x / step) * step
second_data["latBin"] = to_bin(second_data.latitude)
second_data["lonBin"] = to_bin(second_data.longitude)
groups = second_data.groupby(["latBin", "lonBin"])


In [128]:
groups.size().sort_values().value_counts()
# groups.ngroup(ascending=False)+1



1       154448
2        49446
3        23575
4        15112
5         8677
         ...  
71           1
54           1
66           1
64           1
8840         1
Length: 80, dtype: int64